In [1]:
import statsmodels.api as sm
import pandas as pd
import re,string
import nltk
from patsy import dmatrices
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import warnings
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [64]:
df = pd.read_csv("edmunds_comments_20pages.csv",  encoding='utf-8')
df.head()

,date,username,post,quotes
0,September 7,dino001,If they keep it around in next four-five years...,The lease rate is the factor that stops me col...
1,September 7,circlew,The lease rate is the factor that stops me col...,show previous quotes\n\n\nThe Stinger is too b...
2,September 8,qbrozen,circlew said:\nThe lease rate is the factor th...,NaN
3,September 9,FlightNurse2,roadburner said:\nshow previous quotes\n\n\nTh...,"Again, the local dealer are hopeless at best(W..."
4,September 9,roadburner,"Again, the local dealer are hopeless at best(W...","Thanks, but I am more than aware of the distin..."


In [84]:
#clean punctuation and white spaces
df["post_clean"] = df.post.apply(lambda x:re.sub(r"[^a-zA-Z0-9\s]", " ", x.lower()))
df["post_clean"] = df.post_clean.apply(lambda x:re.sub(r"\n", " ", x.lower()))
df["post_clean"] = df.post_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

In [85]:
#removing stop words
stop = set(stopwords.words('english'))
df['post_clean'] = df.post_clean.apply(lambda x: [word for word in x.split() if word not in stop])

In [86]:
df.head(20)

,date,username,post,quotes,post_clean,post_c2
0,September 7,dino001,If they keep it around in next four-five years...,The lease rate is the factor that stops me col...,"[keep, around, next, four, five, years, think,...","[keep, around, next, four, ford, years, think,..."
1,September 7,circlew,The lease rate is the factor that stops me col...,show previous quotes\n\n\nThe Stinger is too b...,"[lease, rate, factor, stops, cold, taking, lea...","[lease, rate, factor, stops, cold, taking, lea..."
2,September 8,qbrozen,circlew said:\nThe lease rate is the factor th...,NaN,"[circlew, said, lease, rate, factor, stops, co...","[circlew, said, lease, rate, factor, stops, co..."
3,September 9,FlightNurse2,roadburner said:\nshow previous quotes\n\n\nTh...,"Again, the local dealer are hopeless at best(W...","[roadburner, said, show, previous, quotes, sti...","[roadburner, said, show, previous, quotes, sti..."
4,September 9,roadburner,"Again, the local dealer are hopeless at best(W...","Thanks, but I am more than aware of the distin...","[local, dealer, hopeless, best, put, ya, car, ...","[local, dealer, hopeless, best, put, ya, car, ..."
5,September 9,FlightNurse2,"roadburner said:\nAgain, the local dealer are ...",I am the poster formerly known as benjaminh an...,"[roadburner, said, local, dealer, hopeless, be...","[roadburner, said, local, dealer, hopeless, be..."
6,September 9,roadburner,"Thanks, but I am more than aware of the distin...",show previous quotes\nI sent you a message,"[thanks, aware, distinction, dealer, group, ow...","[thanks, aware, distinction, dealer, group, ow..."
7,September 9,dino001,"roadburner said:\nThanks, but I am more than a...",NaN,"[roadburner, said, thanks, aware, distinction,...","[roadburner, said, thanks, aware, distinction,..."
8,September 24,benjaminhf,I am the poster formerly known as benjaminh an...,If I do manage to upgrade to a TLX it would ha...,"[poster, formerly, known, benjaminh, benjaminh...","[poster, formerly, known, benjaminh, benjaminh..."
9,September 24,stickguy,Still on the list. Just have not been looking ...,"The fine lrint for all says ""plus dealer add-o...","[still, list, looking, actively, right]","[still, list, looking, actively, right]"


### Replacing model with make in the corpus

In [68]:
#importing model list
car_models = pd.read_csv("car_models_curated_V2.csv", )
car_models.head()
car_models.dtypes

make      object
model1    object
dtype: object

In [69]:
car_models["model1"]=car_models["model1"].astype(str)
car_models["make"] = car_models["make"].apply(lambda x:re.sub(r"[^\w\s]", "", x.lower()))
car_models["model1"] = car_models["model1"].map(lambda x:re.sub(r"[^\w\s]", "", x.lower()))

In [70]:
#converting into dictionary
#This will help in identifying make for corresponding model
car_dict = car_models.set_index('model1').transpose().to_dict()
model = "beetle"
car_dict[model]['make']

C:\Users\abhin\Anaconda2\envs\Python3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until


'volkswagen'

In [71]:
#replace model with make
model1 = set(car_models["model1"])
df["post_c2"] = df.post_clean.map(lambda x: [car_dict[i]['make'] if i in model1 else i for i in x])

In [ ]:
#lemmatizing
#need to get POS for lemmatizing
#create a function that would return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
#wnl = WordNetLemmatizer()
#df['post_lema'] = df.post_clean.apply(lambda x: [wnl.lemmatize(word) for word in x])

### Checking frequency for all values

In [72]:
#getting frequency distribution
car_posts = df['post_c2'].sum()

In [74]:
all_words = nltk.FreqDist(car_posts)
rslt_all = pd.DataFrame.from_dict(all_words,orient='index').reset_index()
rslt_all.columns = ['word','frequency']

,word,frequency
0,keep,16
1,around,22
2,next,22
3,four,17
4,ford,39
5,years,63
6,think,101
7,make,30
8,put,23
9,hazmat,1


### Most popular brands (all)

In [83]:
rslt_all[rslt_all['word'].isin(car_models["make"])].nlargest(10,'frequency')

,word,frequency
33,bmw,239
156,acura,208
212,audi,159
117,polaris,121
13,kia,111
209,honda,110
792,volkswagen,57
159,kymco,51
114,triumph,41
4,ford,39


### Taking unique values (using set)

In [76]:
cars_unique = df.post_c2.map(lambda x:[ "".join(word) for word in sorted(set(x))]).sum() 

In [77]:
unique_words = nltk.FreqDist(cars_unique)
rslt_unique = pd.DataFrame.from_dict(unique_words,orient='index').reset_index()
rslt_unique.columns = ['word','frequency']

,word,frequency
0,340,3
1,440,2
2,appropriate,1
3,around,22
4,behind,9


### Most popular brands (unique)

In [82]:
rslt_unique[rslt_unique['word'].isin(car_models["make"])].nlargest(10,'frequency')

,word,frequency
7,bmw,120
145,acura,83
111,polaris,79
197,audi,78
219,honda,57
33,kia,52
160,kymco,44
816,volkswagen,37
124,triumph,31
101,cub cadet,30


### Calculate Lift Ratio

Identify top 10 brands by frequency. From the posts, calculate lift ratios for associations between the brands. You will have to write a script to do this task). Show the brands on a multi-dimensional scaling (MDS) map (use a Python script for MDS, there are multiple scripts available on the Internet). 

### Insights

### Frequently mentioned attributes for cars

Now pick the 5 most frequently mentioned brands. Which attributes are most strongly associated with which of these 5 brands? 

### While BMW has claimed that they are the “ultimate driving machine”, is that how people feel on Edmunds? Show your analysis.   

### Advice to (i) product manager, and (ii) marketing/advertising manager of these brands based on attribute relations

### Most aspirational brand in your data in terms of people actually wanting to buy or own? Describe your analysis. What are the business implications for this brand?